In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

## Create 3D Gaussians

In [2]:
dim = 3
n_gauss = 3
n_pts_per_gauss = 300
np.random.seed(5)

centers = np.zeros((n_gauss,dim))
for i in range(1,n_gauss):
    centers[i] = np.random.randint(0,2,3)
    
print(centers)

cov_m = [np.diag([0.01 for i in range(dim)]),np.diag([0.1 if i%2 !=0 else 0.1 for i in range(dim)])]

D = np.zeros((n_pts_per_gauss*n_gauss,dim))
c = np.zeros(n_pts_per_gauss*n_gauss)
for i in range(dim):
    k = np.random.randint(0,2,1)[0]
    D[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = np.random.multivariate_normal(centers[i],cov_m[k],n_pts_per_gauss)
    c[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = i
D = (D-np.min(D,axis=0))/(np.max(D,axis=0)-np.min(D,axis=0))
print(D.shape)
print(c.shape)

[[0. 0. 0.]
 [1. 0. 1.]
 [1. 0. 0.]]
(900, 3)
(900,)


## Project 3D to 2D using T-SNE

In [3]:
t_sne = manifold.TSNE(
    n_components=2,
    perplexity=30,
    init="random",
    random_state=0,
)

S = t_sne.fit_transform(D)

In [4]:
# Plotting the t-SNE results with the same color scheme
%matplotlib qt

colors = ['r', 'g', 'b']  # Red, Green, Blue
plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian {i+1}')

plt.title('t-SNE Visualization of 3D Gaussian Distributions into 2D')
plt.legend()
plt.grid(True)
plt.show()

## Create a 2D Grid for Jacobian Calculation

In [5]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

-36.09521 41.842896
-28.966848 41.386196


In [6]:
# Define grid resolution
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
print(yy.shape)
print(y_vals.shape)

(100, 100)
(100,)


In [7]:
%matplotlib qt

plt.figure(figsize=(10, 8))
# Visualize the grid on top of the t-SNE data
plt.scatter(S[:, 0], S[:, 1], c='blue', s=10, label="t-SNE Output")
plt.scatter(xx, yy, c='red', s=5, label="Grid Points")
plt.title("2D t-SNE Output with Grid Points")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
# plt.grid(True)
plt.show()

## Define the Inverse Projection

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# Define the MLP inverse_model
class NNinv(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNinv, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # Input to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 256),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(256, 512),  # Third hidden layer to fourth hidden layer
            nn.ReLU(),
            nn.Linear(512, output_size),  # Fifth hidden layer to output
            nn.Sigmoid()  # Output layer with sigmoid activation
        )
    
    def forward(self, x):
        return self.layers(x)


In [9]:
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(S, D,c, test_size=0.33, random_state=42)

In [10]:
# Example usage
input_size = 2  # Example input size (can be changed)
output_size = dim   # Binary classification (sigmoid output for single output)
inverse_model = NNinv(input_size, output_size)

# Create DataLoader for batch processing
batch_size = 64
t_X_train = torch.tensor(X_train)
t_y_train = torch.tensor(y_train)
dataset = TensorDataset(t_X_train, t_y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the inverse_model, loss function, and optimizer
inverse_model = NNinv(input_size, output_size)
loss_fn = nn.L1Loss()  # Mean Absolute Error (MAE)
optimizer = optim.Adam(inverse_model.parameters(), lr=0.001)

# Number of epochs to train
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        # Forward pass
        outputs = inverse_model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Print the average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete.")

Epoch [1/5], Loss: 0.1066
Epoch [2/5], Loss: 0.0618
Epoch [3/5], Loss: 0.0536
Epoch [4/5], Loss: 0.0522
Epoch [5/5], Loss: 0.0521
Training complete.


In [11]:
t_X_test = torch.tensor(X_test)
t_y_test = torch.tensor(y_test)
outputs_test = inverse_model(t_X_test)
loss_test = loss_fn(outputs_test, t_y_test)
print(loss_test/y_test.shape[0])

tensor(0.0002, dtype=torch.float64, grad_fn=<DivBackward0>)


In [12]:
t_y_test.shape

torch.Size([297, 3])

## Define Jacobian

In [13]:
# Function to compute Jacobian at a specific point
def compute_jacobian(x, y):
    eps = 1e-5  # Small epsilon for numerical differentiation

    # Partial derivatives with respect to x 
    point_hor_plus = torch.tensor([[x + eps, y]]) 
    point_hor_minus = torch.tensor([[x - eps, y]]) 
    f_x_plus_eps = inverse_model(point_hor_plus).detach().numpy()   #3D output
    f_x_minus_eps = inverse_model(point_hor_minus).detach().numpy()
    df_dx = (f_x_plus_eps - f_x_minus_eps) / (2 * eps)

    # Partial derivatives with respect to y
    point_ver_plus = torch.tensor([[x , y + eps]]) 
    point_ver_minus = torch.tensor([[x , y - eps]]) 
    f_y_plus_eps = inverse_model(point_ver_plus).detach().numpy()
    f_y_minus_eps = inverse_model(point_ver_minus).detach().numpy()
    df_dy = (f_y_plus_eps - f_y_minus_eps) / (2 * eps)

    # Jacobian matrix 3x2
    J = np.column_stack((df_dx.T, df_dy.T))
    return J

In [14]:
# Calculate Jacobians over the grid and store results
jacobians = []
for i in range(num_grid_points):
    for j in range(num_grid_points):
        x, y = xx[i, j], yy[i, j]
        jacobian = compute_jacobian(x, y)
        print(jacobian)
        jacobians.append(jacobian)

[[ 0.00298023  0.00093132]
 [-0.0089407   0.00298023]
 [ 0.00447035  0.00074506]]
[[ 0.00242144  0.00186265]
 [-0.0089407   0.00596046]
 [ 0.00335276  0.00186265]]
[[ 0.0031665   0.00149012]
 [-0.0089407   0.00596046]
 [ 0.00409782  0.00186265]]
[[ 0.00204891  0.00204891]
 [-0.0089407   0.00298023]
 [ 0.00335276  0.00149012]]
[[ 0.00335276  0.00149012]
 [-0.00596046  0.00596046]
 [ 0.00558794  0.00186265]]
[[ 0.00279397  0.00111759]
 [-0.00596046  0.00596046]
 [ 0.00558794  0.00149012]]
[[ 0.00223517  0.00242144]
 [-0.00596046  0.00596046]
 [ 0.00409782  0.00298023]]
[[ 0.00242144  0.00298023]
 [-0.00596046  0.0089407 ]
 [ 0.00372529  0.0026077 ]]
[[ 0.00167638  0.00167638]
 [-0.0089407   0.0089407 ]
 [ 0.00372529  0.00111759]]
[[ 0.00279397  0.00186265]
 [-0.0089407   0.00596046]
 [ 0.00186265  0.00186265]]
[[ 0.00279397  0.00279397]
 [-0.00745058  0.00745058]
 [ 0.00447035  0.00335276]]
[[ 0.00204891  0.00279397]
 [-0.00447035  0.00745058]
 [ 0.00223517  0.00447035]]
[[ 0.00298023  0

In [15]:
len(jacobians)


10000

In [16]:

# Convert the list of numpy arrays into a list of PyTorch tensors
jacobian_tensors = [torch.tensor(jacob) for jacob in jacobians]

# Convert the list into a 4D tensor
jacobian_tensor = torch.stack(jacobian_tensors)  # Shape will be [num_grids * num_grids, 3, 2]
jacobian_tensor.shape

# Reshape the tensor to [num_grids, num_grids, 3, 2]
jacobian_tensor_reshaped = jacobian_tensor.view(num_grid_points, num_grid_points, 3, 2)
jacobian_tensor_reshaped.shape

torch.Size([100, 100, 3, 2])

In [17]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np

# # Prepare a combined heatmap data
# # Each Jacobian matrix has a shape of (3, 2)
# combined_heatmap = torch.zeros((num_grid_points * 3, num_grid_points * 2))  # Create an empty array for combined heatmap

# # Fill in the combined heatmap
# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
#         # Fill the appropriate section in the combined heatmap with the Jacobian matrix
#         combined_heatmap[i * 3:(i + 1) * 3, j * 2:(j + 1) * 2] = jacobian_matrix

# # Plot the combined heatmap
# plt.figure(figsize=(10, 10))
# plt.imshow(combined_heatmap, aspect='auto', cmap='viridis', interpolation='nearest')
# plt.title('Combined Jacobian Matrices Heatmap')
# plt.colorbar(label='Value')
# plt.xlabel('Grid Points (2D space)')
# plt.ylabel('Jacobian Matrix Values (3D)')
# # plt.xticks(np.arange(0, num_grid_points * 2, 2), np.arange(num_grid_points))  # X ticks for grid points
# # plt.yticks(np.arange(0, num_grid_points * 3, 3), np.arange(num_grid_points))  # Y ticks for Jacobian matrix rows

# # Show the heatmap
# plt.tight_layout()
# plt.show()


In [18]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.colors import ListedColormap

# # Prepare a combined heatmap data
# combined_heatmap = torch.zeros((num_grid_points * 3, num_grid_points * 2))  # Create an empty array for combined heatmap

# # Fill in the combined heatmap
# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
#         # Fill the appropriate section in the combined heatmap with the Jacobian matrix
#         combined_heatmap[i * 3:(i + 1) * 3, j * 2:(j + 1) * 2] = jacobian_matrix

# # Define a custom colormap with three colors
# colors = ['blue', 'green', 'red']  # Adjust these colors as needed
# cmap = ListedColormap(colors)

# # Normalize the data to fit into the three categories
# # Example: Normalize values to three ranges (low, medium, high)
# combined_heatmap_norm = torch.zeros_like(combined_heatmap)

# # Assuming the values range from a min to a max, you can categorize them
# low_threshold = combined_heatmap.min() + (combined_heatmap.max() - combined_heatmap.min()) / 3
# high_threshold = combined_heatmap.min() + 2 * (combined_heatmap.max() - combined_heatmap.min()) / 3

# for i in range(combined_heatmap.shape[0]):
#     for j in range(combined_heatmap.shape[1]):
#         if combined_heatmap[i, j] < low_threshold:
#             combined_heatmap_norm[i, j] = 0  # Low values
#         elif combined_heatmap[i, j] < high_threshold:
#             combined_heatmap_norm[i, j] = 1  # Medium values
#         else:
#             combined_heatmap_norm[i, j] = 2  # High values

# # Plot the combined heatmap with three colors
# plt.figure(figsize=(10, 10))
# plt.imshow(combined_heatmap_norm, aspect='auto', cmap=cmap, interpolation='nearest')
# plt.title('Combined Jacobian Matrices Heatmap (3 Colors)')
# plt.colorbar(label='Value (Categorized)', ticks=[0, 1, 2], format='%d')
# plt.xlabel('Grid Points (2D space)')
# plt.ylabel('Jacobian Matrix Values (3D)')
# plt.xticks(np.arange(0, num_grid_points * 2, 2), np.arange(num_grid_points))  # X ticks for grid points
# plt.yticks(np.arange(0, num_grid_points * 3, 3), np.arange(num_grid_points))  # Y ticks for Jacobian matrix rows

# # Show the heatmap
# plt.tight_layout()
# plt.show()


## Jacobian norm Heatmap

In [19]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming jacobian_tensor has shape [10, 10, 3, 2]
print("Shape of jacobian_tensor:", jacobian_tensor_reshaped.shape)  # Verify shape

# Calculate the Jacobian norm at each grid point (sum of squared elements in each 3x2 matrix)
jacobian_norms = torch.linalg.matrix_norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()  # shape [num_grid_points, num_grid_points]
jacobian_norms.shape
# Step 2: Plot the Jacobian norm heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(jacobian_norms, xticklabels=False, yticklabels=False, cmap="coolwarm", cbar_kws={'label': 'Jacobian Norm'})
plt.title("Jacobian Norm Heatmap - Approximate Decision Boundaries")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


Shape of jacobian_tensor: torch.Size([100, 100, 3, 2])


#  Contours

In [20]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the Jacobian norm at each grid point
# jacobian_norms = torch.norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()  # Shape [10, 10]
jacobian_norms = torch.linalg.matrix_norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()  # Shape [10, 10]

# Inspect the distribution of norms to decide thresholds
mean_norm = np.mean(jacobian_norms)
std_norm = np.std(jacobian_norms)
print(f"Mean: {mean_norm}, Std Dev: {std_norm}")

# Define threshold levels based on mean and standard deviation
# Use levels that are one standard deviation below and above the mean, but adjust as needed
levels = [mean_norm - std_norm, mean_norm, mean_norm + std_norm]

# Plotting
plt.figure(figsize=(8, 6))
contour = plt.contourf(jacobian_norms, levels=levels, cmap="coolwarm", extend='both')
plt.colorbar(contour, label="Jacobian Norm")
plt.title("Contour Plot with Threshold Levels")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


Mean: 0.023125411942601204, Std Dev: 0.01016931515187025


In [21]:
import torch
import numpy as np
import matplotlib.pyplot as plt

jacobian_norms = torch.norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()

# Use quantiles for levels
q25, q50, q75 = np.percentile(jacobian_norms, [25, 50, 75])
levels_quantiles = [q25, q50, q75]  # You can add more levels if needed

# Normalize jacobian norms (values between 0 and 1)
jacobian_norms_normalized = (jacobian_norms - np.min(jacobian_norms)) / (np.max(jacobian_norms) - np.min(jacobian_norms))

# # Set custom levels based on data range (or normalized data range)
# min_val, max_val = np.min(jacobian_norms), np.max(jacobian_norms)
# # min_val, max_val = np.min(jacobian_norms_normalized), np.max(jacobian_norms_normalized)
# levels_custom = np.linspace(min_val, max_val, num=5)  # Generates 5 evenly spaced levels

# Plot using quantile-based levels
plt.figure(figsize=(8, 6))
contour = plt.contourf(jacobian_norms, levels=levels_quantiles, cmap="coolwarm", extend='both')
# contour = plt.contourf(jacobian_norms, levels=levels_custom, cmap="coolwarm", extend='both')
plt.colorbar(contour, label="Jacobian Norm")
plt.title("Contour Plot with Quantile-Based Levels")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


# Spectral norm

In [22]:
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Calculate spectral norm (largest singular value) for each Jacobian matrix in the grid
# spectral_norms = np.zeros((num_grid_points, num_grid_points))  # Placeholder for storing spectral norms

# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]  # Shape [3, 2]
#         singular_values = torch.linalg.svdvals(jacobian_matrix)  # Get singular values
#         # print(singular_values)
#         spectral_norms[i, j] = singular_values.max().item()  # Largest singular value (spectral norm)

# # Step 2: Plot the heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(spectral_norms, annot=False, cmap="hot", cbar=True)
# plt.title("Spectral Norm Heatmap of Jacobians")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# # plt.colorbar(label="Spectral Norm")
# plt.show()


# SVD on Jacobian

In [23]:
# Initialize lists to store singular values, U and V matrices for each Jacobian
singular_values_list = []
U_matrices = []
V_matrices = []

# Iterate over each grid point and apply SVD
for i in range(jacobian_tensor_reshaped.shape[0]):
    for j in range(jacobian_tensor_reshaped.shape[1]):
        jacobian_matrix = jacobian_tensor_reshaped[i, j]  # Shape: (3, 2)
        
        # Perform SVD
        U, SV, Vt = torch.linalg.svd(jacobian_matrix, full_matrices=False)
        
        # Store the singular values and U, V matrices
        singular_values_list.append(SV)
        U_matrices.append(U)
        V_matrices.append(Vt)

In [24]:
print(U_matrices[0].shape)
print(singular_values_list[0].shape)
print(V_matrices[0].shape)

print(len(U_matrices))
print(len(singular_values_list))
print(len(V_matrices))

torch.Size([3, 2])
torch.Size([2])
torch.Size([2, 2])
10000
10000
10000


In [25]:
import matplotlib.pyplot as plt
import numpy as np

# Reshape singular values to the grid's shape for visualization
# Extract the largest singular value for each point in the grid
largest_singular_values = np.array([s[0] for s in singular_values_list]).reshape(num_grid_points, num_grid_points)


# Plot heatmap
plt.figure(figsize=(8, 6))
plt.imshow(largest_singular_values, cmap='hot', interpolation='nearest')
plt.colorbar(label="Largest Singular Value (Sensitivity)")
plt.xlabel("Grid X")
plt.ylabel("Grid Y")
plt.title("Heatmap of Jacobian Singular Values")
plt.show()


# Visualize U and V

In [26]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming U_matrices is a list of matrices, each of shape (3, 2)
# Convert list of U_matrices to a 3D numpy array for easier manipulation
U_array = np.array(U_matrices)  # Shape should now be (N, 3, 2) if N is the number of matrices

# Determine grid size for visualization based on number of matrices
n_matrices = len(U_matrices)
n_cols = 4  # Choose number of columns for the grid layout
n_rows = (n_matrices + n_cols - 1) // n_cols  # Calculate rows required

# Create a grid of subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))
axes = axes.ravel()  # Flatten to easily iterate over

for i in range(n_matrices):
    im = axes[i].imshow(U_array[i], cmap="viridis", aspect='auto')  # Plot each U matrix
    fig.colorbar(im, ax=axes[i])
    axes[i].set_title(f"U matrix {i+1}")

# Turn off any unused subplots
for j in range(n_matrices, n_rows * n_cols):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming U_matrices and V_matrices are already defined as your 10x10x3 and 10x10x2 grids
# Create the subplot with a flattened array of axes for easy indexing
fig, axes = plt.subplots(3, 2, figsize=(12, 10))
axes = axes.ravel()  # Flatten to a 1D array

# Plot heatmaps for each component of U and V across the grid
for k in range(3):  # 3 components of U (in 3D space)
    im = axes[k].imshow(U_matrices[:, :, k], cmap="viridis")
    fig.colorbar(im, ax=axes[k])
    axes[k].set_title(f"Component {k+1} of U")

for k in range(2):  # 2 components of V (in 2D space)
    im = axes[3 + k].imshow(V_matrices[:, :, k], cmap="plasma")
    fig.colorbar(im, ax=axes[3 + k])
    axes[3 + k].set_title(f"Component {k+1} of V")

plt.tight_layout()
plt.show()


# Note good below

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Assuming jacobian_tensor_reshaped has the shape [10, 10, 3, 2]
# For demonstration, let's create a dummy tensor
# jacobian_tensor_reshaped = torch.rand(10, 10, 3, 2)  # Replace this with your actual reshaped tensor

# Set up the figure and axes
fig, axes = plt.subplots(10, 10, figsize=(15, 15))
fig.suptitle('Jacobian Matrices Heatmap at Different Grid Points', fontsize=16)

# Iterate over the grid points
for i in range(10):
    for j in range(10):
        # Extract the Jacobian matrix at grid point (i, j)
        jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
        # Calculate the norm of the Jacobian for visualization
        jacobian_norm = torch.norm(jacobian_matrix, dim=1).numpy()
        
        # Reshape to a 2D array for heatmap representation (3x2)
        # You can choose to visualize different aspects of the Jacobian as needed
        heatmap_data = jacobian_norm.reshape(3, 1)  # Example to visualize norms
        
        # Plot the heatmap using imshow
        ax = axes[i, j]
        im = ax.imshow(heatmap_data, aspect='auto', cmap='viridis', interpolation='nearest')

        # Set titles and labels
        ax.set_title(f'Point ({i}, {j})')
        ax.set_xticks([])  # Hide x ticks
        ax.set_yticks([])  # Hide y ticks

# Add a colorbar to the right of the figure
cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.95)
cbar.set_label('Norm of Jacobian Vectors')

# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust title positioning
plt.show()


In [21]:
# import torch
# import matplotlib.pyplot as plt

# # Assuming jacobian_tensor_reshaped has the shape [10, 10, 3, 2]
# # For demonstration, create a dummy tensor
# # jacobian_tensor_reshaped = torch.rand(10, 10, 3, 2)  # Replace this with your actual reshaped tensor

# # Set up the figure and axes
# fig, axes = plt.subplots(10, 10, figsize=(15, 15))
# fig.suptitle('Jacobian Matrices at Different Grid Points', fontsize=16)

# # Iterate over the grid points
# for i in range(10):
#     for j in range(10):
#         # Extract the Jacobian matrix at grid point (i, j)
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
#         # Create a heatmap or quiver plot to visualize the Jacobian matrix
#         ax = axes[i, j]
        
#         # Plot the Jacobian matrix using quiver for better understanding of direction
#         ax.quiver(jacobian_matrix[:, 0].numpy(), jacobian_matrix[:, 1].numpy(), angles='xy', scale_units='xy', scale=1, color='r')
        
#         ax.set_xlim(-1, 1)  # Set limits as per your data
#         ax.set_ylim(-1, 1)  # Set limits as per your data
#         ax.set_title(f'Point ({i}, {j})')
#         ax.axis('equal')  # Equal aspect ratio to maintain scale
        
# # Adjust layout
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust title positioning
# plt.show()


In [22]:
# # Convert Jacobians to a more plottable form
# jacobian_x = [j[0, 0] for j in jacobians]
# jacobian_y = [j[1, 1] for j in jacobians]

# # Plot the vector field for the Jacobian magnitudes
# plt.figure(figsize=(10, 8))
# plt.quiver(xx, yy, jacobian_x, jacobian_y, angles="xy", scale_units="xy", scale=1, color='purple')
# plt.title("Jacobian Field (Partial Derivatives Magnitude)")
# plt.xlabel("x")
# plt.ylabel("y")
# # plt.grid(True)
# plt.show()